In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random as rnd

In [2]:
#raw = pd.read_csv('data/2007flightrecords.csv')
raw = pd.read_csv('http://stat-computing.org/dataexpo/2009/2008.csv.bz2')

#Drop all cancelled flights
raw = raw[raw['Cancelled'] == 0]
raw.drop(columns = ['Cancelled'], inplace = True)

raw_samp = raw.iloc[:1000] #Take a small sample to test row manipulation

raw.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Distance,TaxiIn,TaxiOut,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,810,4.0,8.0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,810,5.0,10.0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,515,3.0,17.0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,515,3.0,7.0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,515,3.0,10.0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [3]:
print(raw_samp.columns)
print(raw_samp['UniqueCarrier'].unique())

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')
['WN']


In [4]:
#Set up outcome. Is it more than 30 minutes late?
#raw_samp[['DepTime', 'CRSDepTime','ArrTime', 'CRSArrTime', 'ArrDelay']]

#Luckily, we have the ArrDelay already
#All we have to do is:
raw_samp['Late'] = np.where(raw_samp['ArrDelay'] >= 30, 1, 0)
#print(raw_samp['Late'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
#Set up features and outcomes for the sample
Y_samp = raw_samp['Late']
X_samp = 


SyntaxError: invalid syntax (<ipython-input-5-635b61850a7b>, line 3)